In [5]:
# Import required libraries
import pandas as pd
import dash
#import dash_html_components as html
from dash import html
#import dash_core_components as dcc
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px
from dash import jupyter_dash 

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                    options=[
                                        {'label': 'All Sites', 'value': 'ALL'},
                                        {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                        {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                        {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                        {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                            ],
                                            value='ALL',
                                            placeholder="Select a Launch Site here",
                                            searchable=True
                                             ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                html.Div(dcc.RangeSlider(min=0, max=10000, step=1000, 
                                                            marks={0: '0', 2500: '2500',
                                                            5000: '5000', 7500: '7500', 
                                                            10000: '10000'},
                                                            value=[min_payload, max_payload],
                                                            id='payload-slider')),
                                html.Br(),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    #filtered_df = spacex_df
    if entered_site == 'ALL':
        data_T2_ALL=spacex_df.groupby(['Launch Site'], as_index=False)['class'].mean()
        fig = px.pie(data_T2_ALL, values='class', 
        names='Launch Site', 
        title='Total success launches by site')
        return fig
    else:
        # return the outcomes piechart for a selected site
        data_T2_single=spacex_df.groupby(['Launch Site', 'class'])['class'].count()
        subdf=data_T2_single.loc[entered_site]
        fig = px.pie(subdf, values='class', 
                        names=subdf.index, 
                        title='Total success launches for site ' + entered_site)
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
                [Input(component_id='site-dropdown', component_property='value'), 
                Input(component_id="payload-slider", component_property="value")])
def get_scatter(entered_site, payload):
    print (entered_site, payload)
    if entered_site == 'ALL':
        data_T4_ALL=spacex_df[spacex_df['Payload Mass (kg)'] >= payload[0]]# &
        data_T4_ALL=data_T4_ALL[data_T4_ALL['Payload Mass (kg)'] <= payload[1]]
        fig = px.scatter(data_T4_ALL, 
                            x='Payload Mass (kg)', y='class',
                            color='Booster Version Category',
                            title='correlation between payload and success for ALL sites')
        return fig
    else:
        # return the outcomes for a selected site
        data_T4_single=spacex_df[spacex_df['Launch Site'] == entered_site] 
        data_T4_single=data_T4_single[data_T4_single['Payload Mass (kg)'] >= payload[0]]
        data_T4_single=data_T4_single[data_T4_single['Payload Mass (kg)'] <= payload[1]]             
        print(data_T4_single)
        fig = px.scatter(data_T4_single, 
                            x='Payload Mass (kg)', y='class',
                            color='Booster Version Category',
                            title='correlation between payload and success for site ' + entered_site)
    return fig


# Run the app
if __name__ == '__main__':
    app.run_server(port=8070, jupyter_mode="tab", jupyter_height=500, jupyter_width="70%")   


Dash app running on http://127.0.0.1:8070/


<IPython.core.display.Javascript object>